## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://geemap.org). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [2]:
import ee
import geemap
import pprint

# ee.Authenticate() # Uncomment this line for authorization when running for the first time
ee.Initialize()

## Create an interactive map 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [3]:
import os
os.environ['PROJ_LIB'] = r'/mnt/lustre/a2fs-work3/work/e283/e283/leitm/yhe/venv/miniconda3/envs/jupyter-gee/share/proj'

import sys
sys.path.insert(0, "./lst_cam_modules")

import utils, Landsat_LST
import broadband_emiss as BBE

# geometry = utils.getGeometry('users/yhephd/coords/coord_cambridge')

geo_shp = './data/cambridge/shp/cambridge_district_gcs.shp'
# geo_shp = './data/cambridge/shp/cambridge_wards_gcs.shp'
geometry = geemap.shp_to_ee(geo_shp)

date_start = '2019-01-01';
date_end = '2019-12-31';
use_ndvi = True;
use_cloudmask = False;
cloudlimit = 100;

# compute the LST for each Landsat
# get landsat collection with added variables: NDVI, FVC, TPW, EM, LST
L8coll = Landsat_LST.collection('L8', date_start, date_end, geometry, use_ndvi, use_cloudmask, cloudlimit)
L7coll = Landsat_LST.collection('L7', date_start, date_end, geometry, use_ndvi, use_cloudmask, cloudlimit)
L5coll = Landsat_LST.collection('L5', date_start, date_end, geometry, use_ndvi, use_cloudmask, cloudlimit)
L4coll = Landsat_LST.collection('L4', date_start, date_end, geometry, use_ndvi, use_cloudmask, cloudlimit)

# compute broadband emissivity
L8coll = L8coll.map(BBE.addBand(True))
L7coll = L7coll.map(BBE.addBand(True))
L5coll = L5coll.map(BBE.addBand(True))
L4coll = L4coll.map(BBE.addBand(True))

pp = pprint.PrettyPrinter(depth=3)
pp.pprint(L8coll.getInfo())

{'bands': [],
 'features': [{'bands': [...],
               'id': 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190123',
               'properties': {...},
               'type': 'Image',
               'version': 1563396398193723},
              {'bands': [...],
               'id': 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190224',
               'properties': {...},
               'type': 'Image',
               'version': 1563396580706535},
              {'bands': [...],
               'id': 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190328',
               'properties': {...},
               'type': 'Image',
               'version': 1563396581786059},
              {'bands': [...],
               'id': 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190413',
               'properties': {...},
               'type': 'Image',
               'version': 1563396583178166},
              {'bands': [...],
               'id': 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190429',
               'properties': {...},
   

In [4]:
# select the image
listOfImages = L8coll.toList(L8coll.size())
exImage = ee.Image(listOfImages.get(0))
pp = pprint.PrettyPrinter(depth=1)
pp.pprint(exImage.getInfo())

pp = pprint.PrettyPrinter(depth=1)

vmin = ee.Number(exImage.select('BBE').reduceRegion(
    reducer = ee.Reducer.min(), 
    geometry = geometry, 
    scale = 30, 
    maxPixels = 1e9 
).values().get(0))
pp.pprint('minLST: '+str(vmin.getInfo()))

vmax = ee.Number(exImage.select('BBE').reduceRegion(
    reducer = ee.Reducer.max(),
    geometry = geometry,
    scale = 30,
    maxPixels = 1e9
).values().get(0))
pp.pprint('maxLST: '+str(vmax.getInfo()))

{'bands': [...],
 'id': 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190123',
 'properties': {...},
 'type': 'Image',
 'version': 1563396398193723}
'minLST: 0.9413941592473862'
'maxLST: 0.9887367616721683'


In [5]:
def getband(landsat, bandname):
  def wrap(image):
    return image.select(bandname).rename(bandname + '_' + landsat)
  
  return wrap

# merge all Landsat LST collections for the chart
bandname = 'LST'
LandsatColl = L8coll.map(getband('L8',bandname))
LandsatColl = LandsatColl.merge(L7coll.map(getband('L7',bandname)))
LandsatColl = LandsatColl.merge(L5coll.map(getband('L5',bandname)))
LandsatColl = LandsatColl.merge(L4coll.map(getband('L4',bandname)))
pp = pprint.PrettyPrinter(depth=5)
pp.pprint(LandsatColl.getInfo())

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:32631',
                          'crs_transform': [...],
                          'data_type': {...},
                          'dimensions': [...],
                          'id': 'LST_L8'}],
               'id': 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190123',
               'properties': {'CLOUD_COVER': 59.18,
                              'CLOUD_COVER_LAND': 60.09,
                              'EARTH_SUN_DISTANCE': 0.984214,
                              'ESPA_VERSION': '2_23_0_1b',
                              'GEOMETRIC_RMSE_MODEL': 10.752,
                              'GEOMETRIC_RMSE_MODEL_X': 6.654,
                              'GEOMETRIC_RMSE_MODEL_Y': 8.446,
                              'IMAGE_QUALITY_OLI': 9,
                              'IMAGE_QUALITY_TIRS': 9,
                              'LANDSAT_ID': 'LC08_L1TP_201024_20190123_20190205_01_T1',
                              'LEVEL1_PRODUCTION_DATE': 1549393949000

In [6]:
def func_tvn(image):
  date = ee.Date(image.get('system:time_start'))
  lst = image.select('LST')
  em = image.select('EM')
  fvc = image.select('FVC')
  tpw = image.select('TPW')
  bbe = image.select('BBE')
  return ee.Feature(None, {
    'year':ee.Number(date.get('year')),
    'month':ee.Number(date.get('month')),
    'day':ee.Number(date.get('day')),
    'hour':ee.Number(date.get('hour')),
    'minute':ee.Number(date.get('minute')),
    'lst':ee.Number(lst.reduceRegion(
      ee.Reducer.mean(),geometry,30).get('LST')),
    'tpw':ee.Number(tpw.reduceRegion(
      ee.Reducer.mean(),geometry,30).get('TPW')),
    'em':ee.Number(em.reduceRegion(
      ee.Reducer.mean(),geometry,30).get('EM')),
    'fvc':ee.Number(fvc.reduceRegion(
      ee.Reducer.mean(),geometry,30).get('FVC')),
    'bbe':ee.Number(bbe.reduceRegion(
      ee.Reducer.mean(),geometry,30).get('BBE'))
  })

myFeatures = ee.FeatureCollection(L8coll.map(func_tvn))

pp = pprint.PrettyPrinter(depth=4)
pp.pprint(myFeatures.getInfo())

{'columns': {},
 'features': [{'geometry': None,
               'id': 'LC08_201024_20190123',
               'properties': {'bbe': 0.9722994183905774,
                              'day': 23,
                              'em': 0.9739238833773142,
                              'fvc': 0.09622263340590699,
                              'hour': 10,
                              'lst': -2.774720538263004,
                              'minute': 52,
                              'month': 1,
                              'tpw': 5.782518055555556,
                              'year': 2019},
               'type': 'Feature'},
              {'geometry': None,
               'id': 'LC08_201024_20190224',
               'properties': {'bbe': 0.974986520272739,
                              'day': 24,
                              'em': 0.9764961966277025,
                              'fvc': 0.24499296100295656,
                              'hour': 10,
                              'lst': 15.01

In [7]:
task = ee.batch.Export.table.toDrive(**{
  'collection': myFeatures,
  'description': 'MyTask',
  'folder': 'archer2gee',
  'fileNamePrefix': 'Landsat8_LST_timeseries',
  'fileFormat': 'CSV'
})

task.start()
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: CPFELXRSVB2ZUCACBQGCNVGD).
Polling for task (id: CPFELXRSVB2ZUCACBQGCNVGD).
Polling for task (id: CPFELXRSVB2ZUCACBQGCNVGD).
